In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

def calculate_posterior_params(x, theta_0, phi, phi_0):
    """Calculate posterior mean and standard deviation"""
    precision_sum = 1/phi + 1/phi_0
    posterior_mean = (x/phi + theta_0/phi_0) / precision_sum
    posterior_std = np.sqrt(1/precision_sum)
    return posterior_mean, posterior_std

def calculate_hpd_interval(posterior_mean, posterior_std, alpha=0.05):
    """Calculate the HPD interval for normal distribution"""
    z_value = norm.ppf(1 - alpha/2)
    lower = posterior_mean - z_value * posterior_std
    upper = posterior_mean + z_value * posterior_std
    return lower, upper

def plot_posterior_hpd(x, theta_0, phi, phi_0, alpha=0.05):
    """Plot posterior distribution with HPD interval"""
    # Calculate posterior parameters
    posterior_mean, posterior_std = calculate_posterior_params(x, theta_0, phi, phi_0)
    lower_hpd, upper_hpd = calculate_hpd_interval(posterior_mean, posterior_std, alpha)
    
    # Create range for plotting
    theta_range = np.linspace(posterior_mean - 4*posterior_std,
                            posterior_mean + 4*posterior_std, 1000)
    posterior = norm.pdf(theta_range, posterior_mean, posterior_std)
    
    # Create plot
    plt.figure(figsize=(12, 6))
    plt.plot(theta_range, posterior, 'b-', label='Posterior Distribution')
    
    # Shade HPD region
    hpd_region = np.logical_and(theta_range >= lower_hpd, theta_range <= upper_hpd)
    plt.fill_between(theta_range[hpd_region], posterior[hpd_region], 
                     alpha=0.3, color='blue', label=f'{100*(1-alpha)}% HPD Region')
    
    # Add vertical lines
    plt.axvline(x=posterior_mean, color='r', linestyle='--', 
                label='Posterior Mean')
    plt.axvline(x=lower_hpd, color='g', linestyle=':', label='HPD Bounds')
    plt.axvline(x=upper_hpd, color='g', linestyle=':')
    
    # Add annotations
    plt.title('Posterior Distribution with HPD Interval')
    plt.xlabel('θ')
    plt.ylabel('Density')
    
    # Add text box with calculations
    textstr = f'Posterior Mean: {posterior_mean:.3f}\n'
    textstr += f'Posterior Std: {posterior_std:.3f}\n'
    textstr += f'HPD Interval: [{lower_hpd:.3f}, {upper_hpd:.3f}]'
    plt.text(0.02, 0.98, textstr, transform=plt.gca().transAxes,
             verticalalignment='top', bbox=dict(boxstyle='round', alpha=0.1))
    
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# Example usage
x = 2.0          # observed value
theta_0 = 0.0    # prior mean
phi = 1.0        # likelihood variance
phi_0 = 2.0      # prior variance
alpha = 0.05     # for 95% HPD interval

plot_posterior_hpd(x, theta_0, phi, phi_0, alpha)
print("HPD interval calculation demonstration:")
posterior_mean, posterior_std = calculate_posterior_params(x, theta_0, phi, phi_0)
lower, upper = calculate_hpd_interval(posterior_mean, posterior_std)
print(f"95% HPD interval: [{lower:.3f}, {upper:.3f}]")